### Import Libraries

In [1]:
import nibabel
import ants
import numpy as np
import pandas as pd
import os
from math import ceil
import multiprocessing as mp
import re
import matplotlib.pyplot as plt
import glob

from nilearn.image import load_img, smooth_img, resample_img, get_data
from nilearn.plotting import show, view_img, plot_img

### Define Paths

In [2]:
research_dir = '/media/jraad/Data1/school/research/'
data_dir = os.path.join(research_dir, 'data')
dhcp_dir = os.path.join(data_dir, 'dHCP')
img_dir = os.path.join(dhcp_dir, 'samples')
dhcp_stripped = os.path.join(dhcp_dir, 'stripped')
dhcp_registered = os.path.join(dhcp_dir, 'registered')
dhcp_self_registered = os.path.join(dhcp_dir, 'self_registered')
dhcp_norm = os.path.join(dhcp_dir, 'normalized')
dhcp_self_norm = os.path.join(dhcp_dir, 'self_normalized')
dhcp_standardized = os.path.join(dhcp_dir, 'standardized')

dhcp_remote = '/media/jraad/jadadactyl/dhcp_anat_pipeline'

chm_dir = os.path.join(data_dir, 'chm')
neonates_dir = os.path.join(chm_dir, 'Neonates_T2')
normal_dir = os.path.join(neonates_dir, 'Normal')
abnormal_dir = os.path.join(neonates_dir, 'Abnormal')
death_dir = os.path.join(neonates_dir, 'Death')
chm_stripped = os.path.join(chm_dir, 'Neonates_T2_stripped')
chm_registered = os.path.join(chm_dir, 'registered')
chm_norm = os.path.join(chm_dir, 'normalized')

### Define Uniform Distribution

In [10]:
uniform = np.random.uniform(size=500)
np.save(os.path.join(data_dir, 'uniform.npy'), uniform)

### Read Uniform Distribution

In [3]:
uniform = np.load(os.path.join(data_dir, 'uniform.npy'))

### Define Histogram Normalization

In [5]:
def hist_match(source, template):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image.
    Code adapted from
    http://stackoverflow.com/questions/32655686/histogram-matching-of-two-images-in-python-2-x
    Arguments:
    -----------
        source: np.ndarray
            Image to transform; the histogram is computed over the flattened
            array
        template: np.ndarray
            Template image; can have different dimensions to source
    Returns:
    -----------
        matched: np.ndarray
            The transformed output image
    """

    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
#     s_quantiles /= s_quantiles[-1]
    s_min = min(s_quantiles)
    s_max = max(s_quantiles)
    s_quantiles = (s_quantiles - s_min) / (s_max - s_min)
    
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

### Define Skull Stripping

In [13]:
def strip_image(input_path, output_path):
    stream = os.popen(f'bet2 {input_path} {output_path} -f 0.2')

def process_folder_dhcp(path, output):
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        out_path = os.path.join(output, file)
        strip_image(img_path, out_path)
    
def process_folder_chm(path, output, tag):
    for folder_name in os.listdir(path):
        img_path = os.path.join(path, folder_name, 'T2_anonymous.nii')
        out_path = os.path.join(output, f'{tag}_{folder_name}.nii')
        strip_image(img_path, out_path)

### Read dHCP Metadata

In [21]:
listing = glob.glob(os.path.join(dhcp_remote, 'sub-*'))
counter = 0
for folder in listing:
    subject = os.path.basename(folder)
    if counter == 0:
        session = pd.read_csv(os.path.join(folder,f'{subject}_sessions.tsv'), sep='\t')
        session['subject'] = subject
        session_id = max(session.session_id)
        sessions = session[session.session_id == session_id]
        counter += 1
    else:
        session = pd.read_csv(os.path.join(folder,f'{subject}_sessions.tsv'), sep='\t')
        session.subject = subject
        session['subject'] = subject
        session_id = max(session.session_id)
        sessions = sessions.append(session[session.session_id == session_id])

### Strip dHCP Data

In [28]:
file_list = os.listdir(img_dir)
file_list = sorted(file_list)

for file in file_list:
    subject = file[0:15]
    subject_info = sessions[sessions.subject == subject]
    subject_out = os.path.join(dhcp_stripped, f'{subject}_{subject_info.radiology_score.values[0]}.nii.gz')
    strip_image(os.path.join(img_dir, file), subject_out)

### Strip CHM Data

In [31]:
process_folder(normal_dir, chm_stripped, 'normal')
process_folder(abnormal_dir, chm_stripped, 'abnormal')
process_folder(death_dir, chm_stripped, 'death')

### Clean CHM Data

In [16]:
%%time

x, y, z = (256, 256, 32)

imgs = []
counter = 0
for file in os.listdir(chm_stripped):
    file_path = os.path.join(chm_stripped, file)
    img = ants.image_read(file_path)
    img = ants.resample_image(img, (x, y, z), True, 1)
    img = ants.n3_bias_field_correction(img)
    if counter == 0:
        ants.image_write(img, os.path.join(chm_registered, file))
        ref_img = img
    else:
        img = ants.registration(fixed=ref_img, moving=img, type_of_transform='AffineFast')
        ants.image_write(img['warpedmovout'], os.path.join(chm_registered, file))
    counter += 1

CPU times: user 3min 45s, sys: 5.92 s, total: 3min 51s
Wall time: 32.6 s


### Clean dHCP Data

In [17]:
%%time

samples = []

counter = 0
z_dhcp = 256

file_list = os.listdir(dhcp_stripped)
file_list = sorted(file_list)

for file in file_list:
    print(f"processing image {counter}, {file}")
    file_path = os.path.join(dhcp_stripped, file)
    img = ants.image_read(file_path)    
    img = ants.n3_bias_field_correction(img)
    img = ants.resample_image(img, (x, y, z_dhcp), True, 1)
    for i in range(0, 8):
        sub_path = os.path.join(dhcp_registered, f"set{i}_{file}")
        img_sub = img.numpy()[:,:,range(i,z_dhcp,8)]
        img_sub = ref_img.new_image_like(img_sub)
        img_sub = ants.registration(fixed=ref_img, moving=img_sub, type_of_transform='AffineFast')
        ants.image_write(img_sub['warpedmovout'], sub_path)
    counter += 1

processing image 0, sub-CC00058XX09_1.nii.gz
processing image 1, sub-CC00060XX03_2.nii.gz
processing image 2, sub-CC00062XX05_1.nii.gz
processing image 3, sub-CC00063AN06_1.nii.gz
processing image 4, sub-CC00065XX08_2.nii.gz
processing image 5, sub-CC00066XX09_4.nii.gz
processing image 6, sub-CC00067XX10_1.nii.gz
processing image 7, sub-CC00068XX11_1.nii.gz
processing image 8, sub-CC00069XX12_2.nii.gz
processing image 9, sub-CC00070XX05_3.nii.gz
processing image 10, sub-CC00071XX06_1.nii.gz
processing image 11, sub-CC00072XX07_2.nii.gz
processing image 12, sub-CC00073XX08_1.nii.gz
processing image 13, sub-CC00074XX09_1.nii.gz
processing image 14, sub-CC00075XX10_1.nii.gz
processing image 15, sub-CC00078XX13_2.nii.gz
processing image 16, sub-CC00079XX14_1.nii.gz
processing image 17, sub-CC00080XX07_1.nii.gz
processing image 18, sub-CC00082XX09_1.nii.gz
processing image 19, sub-CC00083XX10_1.nii.gz
processing image 20, sub-CC00084XX11_1.nii.gz
processing image 21, sub-CC00086XX13_2.nii.g

processing image 177, sub-CC00293BN14_2.nii.gz
processing image 178, sub-CC00295XX16_1.nii.gz
processing image 179, sub-CC00298XX19_2.nii.gz
processing image 180, sub-CC00299XX20_1.nii.gz
processing image 181, sub-CC00300XX03_1.nii.gz
processing image 182, sub-CC00301XX04_5.nii.gz
processing image 183, sub-CC00302XX05_1.nii.gz
processing image 184, sub-CC00303XX06_2.nii.gz
processing image 185, sub-CC00304XX07_1.nii.gz
processing image 186, sub-CC00305XX08_2.nii.gz
processing image 187, sub-CC00306XX09_2.nii.gz
processing image 188, sub-CC00307XX10_2.nii.gz
processing image 189, sub-CC00308XX11_1.nii.gz
processing image 190, sub-CC00309BN12_2.nii.gz
processing image 191, sub-CC00313XX08_1.nii.gz
processing image 192, sub-CC00314XX09_2.nii.gz
processing image 193, sub-CC00316XX11_2.nii.gz
processing image 194, sub-CC00319XX14_1.nii.gz
processing image 195, sub-CC00320XX07_2.nii.gz
processing image 196, sub-CC00324XX11_2.nii.gz
processing image 197, sub-CC00326XX13_1.nii.gz
processing im

processing image 352, sub-CC00545XX18_3.nii.gz
processing image 353, sub-CC00546XX19_5.nii.gz
processing image 354, sub-CC00547XX20_3.nii.gz
processing image 355, sub-CC00548XX21_1.nii.gz
processing image 356, sub-CC00549XX22_2.nii.gz
processing image 357, sub-CC00550XX06_1.nii.gz
processing image 358, sub-CC00552XX08_2.nii.gz
processing image 359, sub-CC00553XX09_2.nii.gz
processing image 360, sub-CC00555XX11_2.nii.gz
processing image 361, sub-CC00556XX12_3.nii.gz
processing image 362, sub-CC00558XX14_5.nii.gz
processing image 363, sub-CC00561XX09_1.nii.gz
processing image 364, sub-CC00562XX10_5.nii.gz
processing image 365, sub-CC00563XX11_1.nii.gz
processing image 366, sub-CC00564XX12_2.nii.gz
processing image 367, sub-CC00566XX14_1.nii.gz
processing image 368, sub-CC00568XX16_2.nii.gz
processing image 369, sub-CC00569XX17_2.nii.gz
processing image 370, sub-CC00570XX10_2.nii.gz
processing image 371, sub-CC00571AN11_5.nii.gz
processing image 372, sub-CC00572BN12_2.nii.gz
processing im

### Histogram Match CHM Data

In [18]:
%%time

chm_normal = []
chm_abnormal = []
chm_death = []

for file in os.listdir(chm_registered):
    file_path = os.path.join(chm_registered, file)
    img = ants.image_read(file_path)
    
    img = img.numpy()
    
    img_norm = hist_match(img, uniform)
    
    img_norm = np.rot90(np.rot90(img_norm))
    
    if file.startswith('normal'):
        chm_normal.append(img_norm)
    elif file.startswith('abnormal'):
        chm_abnormal.append(img_norm)
    elif file.startswith('death'):
        chm_death.append(img_norm)

chm_normal = np.array(chm_normal)
chm_abnormal = np.array(chm_abnormal)
chm_death = np.array(chm_death)

np.save(os.path.join(chm_norm, 'chm_normal.npy'), chm_normal)
np.save(os.path.join(chm_norm, 'chm_abnormal.npy'), chm_abnormal)
np.save(os.path.join(chm_norm, 'chm_death.npy'), chm_death)

CPU times: user 4.18 s, sys: 203 ms, total: 4.39 s
Wall time: 5.07 s


### Histogram Match dHCP Data

In [19]:
%%time

dhcp_level1 = []
dhcp_level5 = []

counter = 0

for file in os.listdir(dhcp_registered):
    file_path = os.path.join(dhcp_registered, file)
    
    file_level = file[:-7][-1]
    
    if file_level in ['1', '5']:
        img = ants.image_read(file_path)

        img = img.numpy()

        img_norm = hist_match(img, uniform)
    
        if file_level == '1':
            dhcp_level1.append(img_norm)
        elif file_level == '5':
            dhcp_level5.append(img_norm)

dhcp_level1 = np.array(dhcp_level1)
dhcp_level5 = np.array(dhcp_level5)

np.save(os.path.join(dhcp_norm, 'dhcp_level1.npy'), dhcp_level1)
np.save(os.path.join(dhcp_norm, 'dhcp_level5.npy'), dhcp_level5)

CPU times: user 9min 43s, sys: 28.8 s, total: 10min 12s
Wall time: 11min 32s


## dHCP Only

### Clean dHCP Data

In [10]:
%%time

samples = []

counter = 0
x, y, z = (256, 256, 256)

file_list = os.listdir(dhcp_stripped)
file_list = sorted(file_list)

counter = 0
for file in file_list:
    print(f"processing image {counter}, {file}")
    file_path = os.path.join(dhcp_stripped, file)
    img = ants.image_read(file_path)    
    img = ants.n3_bias_field_correction(img)
    img = ants.resample_image(img, (x, y, z), True, 1)
    if counter == 0:
        ants.image_write(img, os.path.join(dhcp_self_registered, file))
        ref_img = img
    else:
        img = ants.registration(fixed=ref_img, moving=img, type_of_transform='AffineFast')
        ants.image_write(img['warpedmovout'], os.path.join(dhcp_self_registered, file))
    counter += 1

processing image 0, sub-CC00058XX09_1.nii.gz
processing image 1, sub-CC00060XX03_2.nii.gz
processing image 2, sub-CC00062XX05_1.nii.gz
processing image 3, sub-CC00063AN06_1.nii.gz
processing image 4, sub-CC00065XX08_2.nii.gz
processing image 5, sub-CC00066XX09_4.nii.gz
processing image 6, sub-CC00067XX10_1.nii.gz
processing image 7, sub-CC00068XX11_1.nii.gz
processing image 8, sub-CC00069XX12_2.nii.gz
processing image 9, sub-CC00070XX05_3.nii.gz
processing image 10, sub-CC00071XX06_1.nii.gz
processing image 11, sub-CC00072XX07_2.nii.gz
processing image 12, sub-CC00073XX08_1.nii.gz
processing image 13, sub-CC00074XX09_1.nii.gz
processing image 14, sub-CC00075XX10_1.nii.gz
processing image 15, sub-CC00078XX13_2.nii.gz
processing image 16, sub-CC00079XX14_1.nii.gz
processing image 17, sub-CC00080XX07_1.nii.gz
processing image 18, sub-CC00082XX09_1.nii.gz
processing image 19, sub-CC00083XX10_1.nii.gz
processing image 20, sub-CC00084XX11_1.nii.gz
processing image 21, sub-CC00086XX13_2.nii.g

processing image 177, sub-CC00293BN14_2.nii.gz
processing image 178, sub-CC00295XX16_1.nii.gz
processing image 179, sub-CC00298XX19_2.nii.gz
processing image 180, sub-CC00299XX20_1.nii.gz
processing image 181, sub-CC00300XX03_1.nii.gz
processing image 182, sub-CC00301XX04_5.nii.gz
processing image 183, sub-CC00302XX05_1.nii.gz
processing image 184, sub-CC00303XX06_2.nii.gz
processing image 185, sub-CC00304XX07_1.nii.gz
processing image 186, sub-CC00305XX08_2.nii.gz
processing image 187, sub-CC00306XX09_2.nii.gz
processing image 188, sub-CC00307XX10_2.nii.gz
processing image 189, sub-CC00308XX11_1.nii.gz
processing image 190, sub-CC00309BN12_2.nii.gz
processing image 191, sub-CC00313XX08_1.nii.gz
processing image 192, sub-CC00314XX09_2.nii.gz
processing image 193, sub-CC00316XX11_2.nii.gz
processing image 194, sub-CC00319XX14_1.nii.gz
processing image 195, sub-CC00320XX07_2.nii.gz
processing image 196, sub-CC00324XX11_2.nii.gz
processing image 197, sub-CC00326XX13_1.nii.gz
processing im

processing image 352, sub-CC00545XX18_3.nii.gz
processing image 353, sub-CC00546XX19_5.nii.gz
processing image 354, sub-CC00547XX20_3.nii.gz
processing image 355, sub-CC00548XX21_1.nii.gz
processing image 356, sub-CC00549XX22_2.nii.gz
processing image 357, sub-CC00550XX06_1.nii.gz
processing image 358, sub-CC00552XX08_2.nii.gz
processing image 359, sub-CC00553XX09_2.nii.gz
processing image 360, sub-CC00555XX11_2.nii.gz
processing image 361, sub-CC00556XX12_3.nii.gz
processing image 362, sub-CC00558XX14_5.nii.gz
processing image 363, sub-CC00561XX09_1.nii.gz
processing image 364, sub-CC00562XX10_5.nii.gz
processing image 365, sub-CC00563XX11_1.nii.gz
processing image 366, sub-CC00564XX12_2.nii.gz
processing image 367, sub-CC00566XX14_1.nii.gz
processing image 368, sub-CC00568XX16_2.nii.gz
processing image 369, sub-CC00569XX17_2.nii.gz
processing image 370, sub-CC00570XX10_2.nii.gz
processing image 371, sub-CC00571AN11_5.nii.gz
processing image 372, sub-CC00572BN12_2.nii.gz
processing im

### Histogram Match dHCP Data

In [6]:
%%time

dhcp_level1 = []
dhcp_level5 = []

counter = 0

for file in os.listdir(dhcp_self_registered):
    file_path = os.path.join(dhcp_self_registered, file)
    
    file_level = file[:-7][-1]
    
    if file_level in ['1', '5']:
        img = ants.image_read(file_path)

        img = img.numpy()

        img_norm = hist_match(img, uniform)
    
        if file_level == '1':
            dhcp_level1.append(img_norm)
        elif file_level == '5':
            dhcp_level5.append(img_norm)

dhcp_level1 = np.array(dhcp_level1)
dhcp_level5 = np.array(dhcp_level5)

np.save(os.path.join(dhcp_self_norm, 'dhcp_level1.npy'), dhcp_level1)
np.save(os.path.join(dhcp_self_norm, 'dhcp_level5.npy'), dhcp_level5)

CPU times: user 8min 57s, sys: 26.5 s, total: 9min 24s
Wall time: 10min 10s


### dHCP Standardized

In [5]:
%%time

dhcp_level1 = []
dhcp_level5 = []

counter = 0

for file in os.listdir(dhcp_self_registered):
    file_path = os.path.join(dhcp_self_registered, file)
    
    file_level = file[:-7][-1]
    
    if file_level in ['1', '5']:
#         print(f"Processing image {counter}")
        img = ants.image_read(file_path)

        img = img.numpy()

        amax = np.amax(img)
        amin = np.amin(img)
        
        img = (img - amin) / (amax - amin)
    
        if file_level == '1':
            dhcp_level1.append(img)
        elif file_level == '5':
            print(file)
            dhcp_level5.append(img)
            
        counter += 1

# dhcp_level1 = np.array(dhcp_level1)
# dhcp_level5 = np.array(dhcp_level5)

# np.save(os.path.join(dhcp_standardized, 'dhcp_level1.npy'), dhcp_level1)
# np.save(os.path.join(dhcp_standardized, 'dhcp_level5.npy'), dhcp_level5)

sub-CC00087AN14_5.nii.gz
sub-CC00103XX04_5.nii.gz
sub-CC00194XX14_5.nii.gz
sub-CC00218AN12_5.nii.gz
sub-CC00231XX09_5.nii.gz
sub-CC00248XX18_5.nii.gz
sub-CC00255XX08_5.nii.gz
sub-CC00301XX04_5.nii.gz
CPU times: user 28.8 s, sys: 5.96 s, total: 34.7 s
Wall time: 35 s
